# Subsistema de Privacidad y Seguridad basado en Redis

Este notebook implementa el Subsistema de Privacidad y Seguridad de Duolingo utilizando Redis como motor principal de almacenamiento para datos sensibles, auditoría, control de acceso y operaciones efímeras. La implementación sigue la arquitectura planteada en la entrega, priorizando baja latencia, trazabilidad, consistencia configurable y la aplicación del principio de mínimo privilegio.

Redis se ejecuta localmente mediante Docker Compose, con el puerto 6379 mapeado para permitir el acceso desde esta notebook.

---

## Objetivo del Notebook

El objetivo es demostrar los principales patrones de acceso del subsistema, cada uno implementado utilizando las primitivas de Redis adecuadas.

### 1. Validación de tokens y control de acceso
Incluye:
- Conjuntos (Sets) para representar roles y permisos (RBAC)
- Strings con TTL para tokens de sesión
- Marcas de revocación de tokens
- Streams de auditoría para registrar validaciones y accesos

### 2. Actualización de preferencias y gestión de consentimientos
Incluye:
- Almacenamiento de preferencias en formato JSON
- Historial de consentimientos mediante listas
- Registro automático de auditoría ante cada modificación

### 3. Registro y consumo de eventos de auditoría
Incluye:
- Stream global de auditoría (`audit:events`)
- Stream individual por usuario (`audit:byUser:{userId}`)
- Consultas cronológicas y reversas

### 4. Operaciones efímeras con expiración automática
Incluye:
- Uso de TTL para asegurar eliminación automática
- Datos temporales como tokens o claves de recuperación

### 5. Eliminación y anonimización asincrónica
Incluye:
- Uso de streams como colas de trabajo
- Simulación de un proceso worker que elimina o anonimiza datos
- Registro de auditoría de cada operación ejecutada

---

# Modelo de Datos y Namespaces

El subsistema organiza la información mediante claves jerárquicas con el formato:

`<dominio>:<subdominio>:<identificador>[:atributo]`

Ejemplos de claves utilizadas:

- `sec:role:{roleId}` → conjunto de permisos asociados a un rol  
- `sec:userRoles:{userId}` → conjunto de roles asignados a un usuario  
- `sec:token:{jti}` → token de sesión con TTL  
- `sec:revoked:{jti}` → marca de token revocado  
- `privacy:prefs:{userId}` → preferencias de privacidad del usuario  
- `privacy:consent:{userId}` → historial de consentimientos  
- `audit:events` → stream global de auditoría  
- `audit:byUser:{userId}` → stream por usuario  
- `privacy:deleteQueue` → cola de solicitudes de eliminación o anonimización  

Este esquema facilita la separación de responsabilidades, la aplicación de políticas diferenciadas de expiración y la trazabilidad completa de operaciones.

---

# Archivo auxiliar

Para mantener la notebook organizada y modular, toda la lógica del subsistema se implementa en el archivo auxiliar `security_setup.py`.

Este archivo define:

- Funciones para crear roles, permisos y asignar roles a usuarios
- Lógica de RBAC y validación de permisos
- Emisión, validación y revocación de tokens con TTL
- Gestión de preferencias de privacidad y consentimientos
- Registro y consulta de eventos mediante Redis Streams
- Encolado y procesamiento de solicitudes de eliminación y anonimización
- Conexión automática a Redis tanto dentro como fuera de Docker

La notebook importa estas funciones y se concentra exclusivamente en demostrar los patrones de acceso y la persistencia de los datos según la configuración de Redis.




In [ ]:
%pip install redis

# Conexión con Redis

En esta celda conectamos contra el servicio `redis` definido en `docker-compose.yml`.
Esta celda debe ejecutarse siempre antes de correr cualquier prueba.

In [ ]:
from security_setup import *
r = connect("redis")
print("Conectado:", r.ping())

## Patrón: Validación de Tokens + Control de Acceso (RBAC)

Esta celda crea:
- Roles (`student`, `moderator`)
- Permisos asociados
- Relaciones usuario–rol
- Emisión de un token de sesión con TTL

Se dejará un JTI para que pueda verificarse más adelante
(evaluando así persistencia según Redis esté configurado con RDB/AOF).

In [ ]:
add_permission_to_role(r, "student", "user:view_profile")

add_permission_to_role(r, "moderator", "audit:view")

assign_role_to_user(r, "user:1", "student")
assign_role_to_user(r, "user:2", "moderator")

jti = issue_token(r, "user:1", ttl_seconds=120)
print("Token emitido:", jti)

last_jti = jti

## Lectura y verificación del patrón RBAC + Tokens

Esta celda:
- Verifica el rol del usuario
- Lista permisos
- Comprueba si tiene o no un permiso específico
- Valida el token emitido anteriormente

Permite testear si la persistencia de Redis conserva los datos tras reiniciar kernel/Redis.


In [ ]:
print("Roles user:1:", get_user_roles(r, "user:1"))
print("Permisos del rol student:", get_role_permissions(r, "student"))

print("user:1 puede ver perfil? =>", user_has_permission(r, "user:1", "user:view_profile"))
print("user:1 puede ver auditoría? =>", user_has_permission(r, "user:1", "audit:view"))

print("Validación del token:", validate_token(r, last_jti))

## Patrón: Actualización de Preferencias y Gestión de Consentimientos

En esta celda se guardan:
- Preferencias de privacidad (`privacy:prefs:{userId}`)
- Consentimientos históricos (`privacy:consent:{userId}`)

Cada operación genera un evento de auditoría.


In [ ]:
prefs_u1 = {
    "profile_visibility": "friends_only",
    "share_learning_stats": True,
    "share_location": False,
}

set_privacy_prefs(r, "user:1", prefs_u1)

add_consent_entry(r, "user:1", "terms_and_conditions", True)
add_consent_entry(r, "user:1", "marketing_emails", False)

print("Preferencias creadas para user:1")

## Lectura de Preferencias y Consentimientos

Permite verificar persistencia:
- Las preferencias deben recuperarse intactas
- La lista de consentimientos debe reflejar el historial

In [ ]:
print("Preferencias persistidas:", get_privacy_prefs(r, "user:1"))

print("\nConsentimientos guardados:")
for raw in r.lrange(key_privacy_consent("user:1"), 0, -1):
    print("-", json.loads(raw))

## Patrón: Registro de Auditoría con Redis Streams

Se generan eventos artificiales para demostrar:
- Registro global en `audit:events`
- Registro específico por usuario en `audit:byUser:{userId}`

In [ ]:
add_audit_event(r, "user:1", "privacy_view", "success", {"section": "profile"})
add_audit_event(r, "user:2", "privacy_update", "success", {"field": "visibility"})
add_audit_event(r, "user:1", "token_validation", "failure", {"reason": "expired"})

print("Eventos de auditoría generados.")

## Lectura de Auditoría (Global y Por Usuario)

Se leen:
- Los últimos N eventos del stream global
- Los eventos correspondientes al usuario `user:1`

In [ ]:
print("Últimos eventos globales:")
for event_id, fields in get_last_audit_events(r, 10):
    print(event_id, "=>", fields)

print("\nÚltimos eventos de user:1:")
for event_id, fields in get_last_audit_events_by_user(r, "user:1", 10):
    print(event_id, "=>", fields)

## Patrón: Claves Efímeras con TTL

Aquí creamos un token que expira en 10 segundos.  
Redis lo borrará automáticamente.

In [ ]:
jti_ttl = issue_token(r, "user:3", ttl_seconds=10)
print("Token efímero emitido:", jti_ttl)

## Verificación de Expiración

Se muestra:
- El TTL restante
- Si la clave existe
- Si la validación falla una vez expirado

In [ ]:
print("TTL restante:", r.ttl(key_token(jti_ttl)))
print("Token existe?", r.exists(key_token(jti_ttl)))
print("Validación:", validate_token(r, jti_ttl))

## Patrón: Anonimización Asíncrona

Se encola una solicitud en `privacy:deleteQueue`.  
Un worker (simulado) la procesará luego.

In [ ]:
enqueue_delete_request(r, "user:1", "right_to_be_forgotten")
print("Solicitud encolada.")

## Procesamiento de Solicitudes de Eliminación

Esta celda ejecuta el “worker” y luego verifica:
- roles eliminados
- preferencias eliminadas
- consentimientos removidos

Todo queda registrado también en el stream de auditoría.

In [ ]:
processed = process_delete_requests(r)
print("Procesado:", processed)

print("\nEstado de user:1 luego de anonimización:")
print("Roles:", get_user_roles(r, "user:1"))
print("Preferencias:", get_privacy_prefs(r, "user:1"))
print("Consentimientos:", r.lrange(key_privacy_consent("user:1"), 0, -1))

## Conclusiones y relación con los requerimientos no funcionales

A partir de los ejemplos anteriores se ve cómo Redis cumple los puntos de la letra:

1. **Baja latencia y alto rendimiento**  
   - Todas las operaciones clave-valor y de sets se realizan en memoria.  
   - Validación de tokens (`GET`, `EXISTS`) y verificación de permisos (`SISMEMBER`) son O(1).

2. **Manejo de datos efímeros y expiración automática**  
   - Tokens de sesión (`sec:token:{jti}`) se crean con TTL (opción `ex` de `SET`).  
   - Flags de revocación (`sec:revoked:{jti}`) usan el mismo TTL para sincronizar expiración.

3. **Seguridad y cifrado (a nivel conceptual)**  
   - En producción, Redis puede configurarse con TLS, ACLs por usuario y autenticación.  
   - La notebook se centra en la parte de modelo y patrones de acceso.

4. **Consistencia y durabilidad configurables**  
   - Redis puede combinar RDB/AOF para persistir los datos críticos
     (por ejemplo, Streams de auditoría y configuraciones de privacidad).

5. **Trazabilidad y auditoría continua**  
   - Cada acción relevante (emisión y validación de tokens, cambios de preferencias,
     solicitudes de borrado) queda registrada en `audit:events` y `audit:byUser:{userId}`.  
   - Redis Streams permiten procesar estos eventos en tiempo real o de forma diferida
     sin bloquear las operaciones de la aplicación.

6. **Procesos asíncronos y no bloqueantes**  
   - La cola `privacy:deleteQueue` permite gestionar solicitudes de anonimización de forma asíncrona,
     evitando afectar la experiencia del resto de los usuarios.

En conjunto, este subsistema ilustra cómo Redis puede actuar como **motor central de seguridad,
privacidad y auditoría**, cumpliendo con los principios de confidencialidad, integridad,
disponibilidad y trazabilidad exigidos por la plataforma.
